## Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [31]:
import pandas as pd

**Вариант 1**

In [32]:
movies_data = pd.read_csv('ml-latest-small/ratings.csv')
movies_data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [33]:
movies_data_ = movies_data.groupby('userId').agg({'rating':'count','timestamp':['min','max']})
movies_data_ = movies_data_[movies_data_['rating']['count']>100]
movies_data_.head()

rating   timestamp            
        count         min         max
userId                               
4         204   949778714   949982274
8         116  1154389340  1154474527
15       1700   997937239  1469330735
17        363  1127468587  1127476640
19        423   855190091   855195373

In [34]:
def find_diff(row):
    return row['timestamp']['max'] - row['timestamp']['min']

In [35]:
movies_data_['lifetime'] = movies_data_.apply(find_diff,axis=1)
print(f"Cреднее время жизни пользователей, которые выставили более 100 оценок: {movies_data_['lifetime'].mean()}")

Cреднее время жизни пользователей, которые выставили более 100 оценок: 40080507.4496124


### Как перевести это неудобоваримое число в дни, часы или что-то подобное? Можно ли короче, чем вариант 2? 

**Вариант 2**

In [36]:
movies_data = pd.read_csv('ml-latest-small/ratings.csv')
movies_data['date'] = pd.to_datetime(movies_data['timestamp'], unit='s')
movies_data.head()

,userId,movieId,rating,timestamp,date
0,1,31,2.5,1260759144,2009-12-14 02:52:24
1,1,1029,3.0,1260759179,2009-12-14 02:52:59
2,1,1061,3.0,1260759182,2009-12-14 02:53:02
3,1,1129,2.0,1260759185,2009-12-14 02:53:05
4,1,1172,4.0,1260759205,2009-12-14 02:53:25


In [37]:
movies_data_ = movies_data.groupby('userId').agg({'rating':'count','date':['min','max']})
movies_data_ = movies_data_[movies_data_['rating']['count']>100]
movies_data_.head()

rating                date                    
        count                 min                 max
userId                                               
4         204 2000-02-05 19:25:14 2000-02-08 03:57:54
8         116 2006-07-31 23:42:20 2006-08-01 23:22:07
15       1700 2001-08-16 04:47:19 2016-07-24 03:25:35
17        363 2005-09-23 09:43:07 2005-09-23 11:57:20
19        423 1997-02-06 00:48:11 1997-02-06 02:16:13

In [38]:
def find_diff(row):
    return row['date']['max'] - row['date']['min']

In [39]:
movies_data_['lifetime'] = movies_data_.apply(find_diff,axis=1)
# movies_data_.head()
print(f"Cреднее время жизни пользователей, которые выставили более 100 оценок: {movies_data_['lifetime'].mean()}")

Cреднее время жизни пользователей, которые выставили более 100 оценок: 463 days 21:28:27.449612


## Задание 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [62]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [63]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [58]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [59]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


Необходимо сформировать две таблицы:
1. таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
1. аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [70]:
(
    rzd
    .merge(auto, how='outer', on='client_id')
    .merge(air, how='outer', on='client_id')
)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [72]:
(
    client_base
    .merge(rzd, how='outer', on='client_id')
    .merge(auto, how='outer', on='client_id')
    .merge(air, how='outer', on='client_id')
    
)

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


## Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т.е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
1. У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
1. Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах п.1?
1. Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

1. Можно отслеживать факт покупки по изменению баланса, если речь идёт о банк-клиенте, к примеру. Можно проверять баланс между двумя смежными местоположениями. Связь с таблицой покупок и визитов может быть установлена по времени. Это наиболее корректный вариант. Можно также устанавливать связь по категории покупок. К примеру, если известна категория конкретной покупки и продукты данной категории могут быть приобретены в магазине, координаты которого совпадают с координатами местоположения клиента, то может быть установлено соответствие "координаты-покупка". Однако этот вариант неточный, так как товары одной категории могут быть в различных магазинах, которые посещал клиент и однозначного соответствия установить не удастся.  
1. Местоположения должны быть отсортированы в хронологическом порядке. В противном случае отслеживание движения финансовых потоков будет затруднено.  
1. Популярность приложения/сервиса в том или ином регионе. Можно выделить регионы с меньшей плотностью покупок, но высокой посещаемостью как потенциально перспективные для продвижения приложения/сервиса, т.к. вполне возможно в этих регионах есть спрос на предлагаемый товар/услугу, но есть какие-то технические проблемы с его доставкой, требуется корректировка ценовой политики и т.д.